<h2 style="color:crimson">Tarea 5</h2>

1. Repite el ejemplo *Arxiv ML Papers (I)*, agrega dos nuevas columnas al DataFrame final: `Abstract`, con el resumen de cada paper; `Submitted`, con la fecha en la que fue agregado al sistema. **Hint**: Ambos elementos se encuentran dentro de la *url* encontrada.

----

2. Por medio de la librería `requests` y el API de AlphaVantage, consigue los precios históricos de las monedas `["MXN", "EUR", "BRL", "ARS", "CHF"]` respecto al dólar (todos los precios deben estar en dólares). Crea un DataFrame con cada colúmna la moneda de referencia:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>MXN</th>      <th>EUR</th>      <th>BRL</th>      <th>ARS</th>      <th>CHF</th>    </tr>  </thead>  <tbody>    <tr>      <th>2014-11-24</th>      <td>0.0732</td>      <td>1.2438</td>      <td>0.3920</td>      <td>0.1169</td>      <td>1.0342</td>    </tr>    <tr>      <th>2014-11-25</th>      <td>0.0731</td>      <td>1.2473</td>      <td>0.3955</td>      <td>0.1169</td>      <td>1.0370</td>    </tr>    <tr>      <th>2014-11-26</th>      <td>0.0728</td>      <td>1.2507</td>      <td>0.3995</td>      <td>0.1168</td>      <td>1.0401</td>    </tr>    <tr>      <th>2014-11-27</th>      <td>0.0726</td>      <td>1.2458</td>      <td>0.3947</td>      <td>0.1169</td>      <td>1.0363</td>    </tr>    <tr>      <th>2014-11-28</th>      <td>0.0717</td>      <td>1.2451</td>      <td>0.3891</td>      <td>0.1168</td>      <td>1.0351</td>    </tr>  </tbody></table>


----
3. Con el DataFrame creado en el ejercicio 2, crea el siguiente mapa de calor con la correlación entre cada una de las monedas. **hint**: Revisa que métodos tiene un DataFrame para crear correlaciones; considera la librería `seaborn`.

![](../files/lec04/imgs/heatmap.png)

---

4. Con el DataFrame creado en el ejercicio 2, crea un nuevo DataFrame que incluya una correlación móvil con ventana de 30 días de cada una de las monedas respecto a `"MXN"`. **hint**: Investiga sobre el método `.xs` de un DataFrame

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>EUR</th>      <th>BRL</th>      <th>ARS</th>      <th>CHF</th>    </tr>  </thead>  <tbody>    <tr>      <th>2015-03-19</th>      <td>0.900941</td>      <td>0.870440</td>      <td>0.835029</td>      <td>0.916637</td>    </tr>    <tr>      <th>2015-03-20</th>      <td>0.874866</td>      <td>0.815453</td>      <td>0.762308</td>      <td>0.903594</td>    </tr>    <tr>      <th>2015-03-22</th>      <td>0.866783</td>      <td>0.789977</td>      <td>0.726098</td>      <td>0.900547</td>    </tr>    <tr>      <th>2015-03-23</th>      <td>0.835823</td>      <td>0.728781</td>      <td>0.643129</td>      <td>0.885424</td>    </tr>    <tr>      <th>2015-03-24</th>      <td>0.801267</td>      <td>0.671188</td>      <td>0.560149</td>      <td>0.877682</td>    </tr>  </tbody></table>

---

5. Con el DataFrame creado en el ejercicio 4, crea la siguiente figura con _scatterplots_ entre todas las combinaciones de las correlaciones respecto a `"MXN"`. ¿Encuentras algún patrón en alguna combinación?

![](../files/lec04/imgs/scatter.png)

---

6. Con el DataFrame creado en el ejercicio 4, crea la siguiente figura, la cuál representa, a cada observación, la correlación promedio entre todas las monedas respecto a `"MXN"` y unas bandas que representan la correlación máxima y mínima a cada observación.

![](../files/lec04/imgs/corr_ts.png)

In [147]:
from requests_html import HTML    
from requests_html import HTMLSession
import pandas as pd
import os
import requests
import matplotlib.pyplot as plt

<h2>Pregunta 1</h2>

In [131]:
url = "https://arxiv.org/list/stat.ML/recent"
with HTMLSession() as sess:
    r = sess.get(url)
r

<Response [200]>

In [132]:
divpage = r.html.find("div[id='dlpage']", first=True)
papers_links = divpage.find("dt")
papers_titles = divpage.find("dd")
link_secc=papers_links[0]
title_secc=papers_titles[0]

In [133]:
values = []
for link_secc, title_secc in zip(papers_links, papers_titles):
    title = title_secc.find("div[class='list-title mathjax']", first=True).text
    
    abstract_url = link_secc.find("a[title='Abstract']", first=True).absolute_links
    abstract_url, *_ = abstract_url
    
    values.append([title, abstract_url])

In [134]:
abstracts = []
for title, abstract_url in values:
    with HTMLSession() as sess:
        r_abstract = sess.get(abstract_url)
        div_abs = r_abstract.html.find("div.leftcolumn", first=True) 
        abstract = div_abs.find("blockquote", first=True).text.strip() if div_abs else "No abstract found"
        abstracts.append(abstract)

In [135]:
import re

dates = []
for title, abstract_url in values:
    with HTMLSession() as sess:
        r_d = sess.get(abstract_url)
        div_d = r_d.html.find("div.leftcolumn", first=True) 
        d_text = div_d.find("div.submission-history", first=True).text.strip()
        date_match = re.search(r'\b(\d{1,2}\s+\w+\s+\d{4})\b', d_text)
        date = date_match.group(1) if date_match else "Date not found"
        dates.append(date)

In [136]:
df_values = pd.DataFrame(values, columns=["title", "url"])
df_values["abstract"] = abstracts
df_values["submitted"] = dates
df_values["title"] = df_values["title"].str.replace("Title: ", "")  
df_values["abstract"] = df_values["abstract"].str.replace("Abstract:", "")  
df_values

,title,url,abstract,submitted
0,A prediction rigidity formalism for low-cost u...,https://arxiv.org/abs/2403.02251,Regression methods are fundamental for scienti...,4 Mar 2024
1,Recency-Weighted Temporally-Segmented Ensemble...,https://arxiv.org/abs/2403.02150,Time-series modeling in process industries fac...,4 Mar 2024
2,Differential Privacy of Noisy (S)GD under Heav...,https://arxiv.org/abs/2403.02051,Injecting heavy-tailed noise to the iterates o...,4 Mar 2024
3,Bipartite Graph Variational Auto-Encoder with ...,https://arxiv.org/abs/2403.02011,We propose a method to represent bipartite net...,4 Mar 2024
4,Capacity of the Hebbian-Hopfield network assoc...,https://arxiv.org/abs/2403.01907,"In \cite{Hop82}, Hopfield introduced a \emph{H...",4 Mar 2024
5,Improving generalisation via anchor multivaria...,https://arxiv.org/abs/2403.01865,We introduce a causal regularisation extension...,4 Mar 2024
6,Soft-constrained Schrodinger Bridge: a Stochas...,https://arxiv.org/abs/2403.01717,Schrödinger bridge can be viewed as a continuo...,4 Mar 2024
7,CATS: Enhancing Multivariate Time Series Forec...,https://arxiv.org/abs/2403.01673,For Multivariate Time Series Forecasting (MTSF...,4 Mar 2024
8,Sample Efficient Myopic Exploration Through Mu...,https://arxiv.org/abs/2403.01636,Multitask Reinforcement Learning (MTRL) approa...,3 Mar 2024
9,Limits to classification performance by relati...,https://arxiv.org/abs/2403.01571,The performance of machine learning classifica...,3 Mar 2024


<h2>Pregunta 2</h2>

In [155]:
os.environ['ALPHA_VANTAGE_KEY']='6P5VDEKZY1GEBDN2'
avkey=os.environ['ALPHA_VANTAGE_KEY']
url = "https://www.alphavantage.co"
base_url = "https://www.alphavantage.co/query"

In [156]:
currencies = ["MXN", "EUR", "BRL", "ARS", "CHF"]
df_data = pd.DataFrame()

# Iterar sobre las monedas y hacer la solicitud a la API
for currency in currencies:
    intraday_params = {
        "function": "FX_DAILY",
        "from_symbol": currency,
        "to_symbol": "USD",
        "apikey": avkey,
        "outputsize": "full"
    }

    r = requests.get(base_url, params=intraday_params)
    data = r.json()

    if "Time Series FX (Daily)" in data:
        df_curr = pd.DataFrame(data["Time Series FX (Daily)"]).T
        df_curr.index = pd.to_datetime(df_curr.index)
        df_curr[f"{currency}_close"] = df_curr["4. close"].astype(float)
        df_data = pd.concat([df_data, df_curr[f"{currency}_close"]], axis=1)

df_data.sort_index(inplace=True)

print(df_data.head())


                     MXN_close  EUR_close  BRL_close  ARS_close  CHF_close
2005-01-05 00:00:00        NaN     1.3258        NaN        NaN        NaN
2005-01-06 00:00:00        NaN     1.3172        NaN        NaN        NaN
2005-01-07 00:00:00        NaN     1.3050        NaN        NaN        NaN
2005-01-10 00:00:00        NaN     1.3075        NaN        NaN        NaN
2005-01-11 00:00:00        NaN     1.3106        NaN        NaN        NaN


<h2>Pregunta 3</h2>

In [148]:
import seaborn as sns

## References


* https://developer.mozilla.org/en-US/docs/Web
* https://www.w3schools.com/tags/ref_httpmethods.asp